DAMAS Homework 2 - Vicki Seely (vicki.seely@target.com)

HW2.0:
How do you merge  two sorted  lists/arrays of records of the form [key, value]? Where is this  used in Hadoop MapReduce? [Hint within the shuffle]
What is  a combiner function in the context of Hadoop? 
Give an example where it can be used and justify why it should be used in the context of this problem.
What is the Hadoop shuffle?

In [1]:
lista = [1,2,3,4]
listb = [5,6,7,8]
merge_list = lista + listb
print merge_list

[1, 2, 3, 4, 5, 6, 7, 8]


** List/Array merging is used by MapReduce to combine results from mappers to combiners and from combiners to reducers. The combiner takes results from the mapper and essentially does a pre-sort/pre-aggregation in preparation of sending the key-value pairs to the reducer. A combiner is always a good idea. ;-) It can be used when aggregating word counts - especially when you're dealing with big data as it reduces the amount of time needed by the reducer to process.

HW2.1: Counters as a debugging aid (and for getting work done, but please use sparingly as they are heavy)

Consumer complaints dataset: Use Counters to do EDA (exploratory data analysis and to monitor progress)
Counters are lightweight objects in Hadoop that allow you to keep track of system progress in both the map and reduce stages of processing. By default, Hadoop defines a number of standard counters in "groups"; these show up in the jobtracker webapp, giving you information such as "Map input records", "Map output records", etc. 

While processing information/data using MapReduce job, it is a challenge to monitor the progress of parallel threads running across nodes of distributed clusters. Moreover, it is also complicated to distinguish between the data that has been processed and the data which is yet to be processed. The MapReduce Framework offers a provision of user-defined Counters, which can be effectively utilized to monitor the progress of data across nodes of distributed clusters.

Use the Consumer Complaints  Dataset provide here to complete this question:

     https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0

Use the following command to grab the file:

    curl -L https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0 -o Consumer_Complaints.csv

The consumer complaints dataset consists of diverse consumer complaints, which have been reported across the United States regarding various types of loans. The dataset consists of records of the form:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?

Here’s is the first few lines of the  of the Consumer Complaints  Dataset:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,
1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
1114255,Bank account or service,Checking account,Deposits and withdrawals,,NY,11102,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, Inc.)",In progress,Yes,
1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,

User-defined Counters

Now, let’s use MapReduce Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories (all other categories get lumped into this one) in the consumer complaints dataset. Basically produce the distribution of the Product column in this dataset using counters (limited to 3 counters here).

Hadoop offers Job Tracker, an UI tool to determine the status and statistics of all jobs. Using the job tracker UI, developers can view the Counters that have been created. Screenshot your  job tracker UI as your job completes and include it here. Make sure that your user defined counters are visible. 


In [2]:
!curl -L https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0 -o Consumer_Complaints.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 48.5M  100 48.5M    0     0  11.7M      0  0:00:04  0:00:04 --:--:-- 15.5M


In [3]:
%%writefile vjsCC_cnt.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re
 
class vjsCC(MRJob):
    
    def mapper(self, _, line):
        line = line.decode('utf8', 'ignore')
        id,grp,_ = line.split(",",2)
        if grp.lower() != 'product':
            if grp == 'Debt collection':
                self.increment_counter('type','Debt collection',1)
            elif grp == 'Mortgage':
                self.increment_counter('type','Mortgage',1)
            else:
                self.increment_counter('type','Other',1)
            
    def reducer(self, grp, counts):
        yield grp, sum(counts)

if __name__ == '__main__':
    vjsCC.run()

Overwriting vjsCC_cnt.py


In [4]:
%reload_ext autoreload
%autoreload 2
from vjsCC_cnt import vjsCC
mr_job = vjsCC(args=['Consumer_Complaints.csv'])
with mr_job.make_runner() as runner: 
    runner.run()
    print runner.counters()

[{'type': {'Debt collection': 44372, 'Other': 142788, 'Mortgage': 125752}}]


In [5]:
!aws emr list-clusters

{
    "Clusters": [
        {
            "Status": {
                "Timeline": {
                    "ReadyDateTime": 1466003662.716, 
                    "CreationDateTime": 1466003182.517
                }, 
                "State": "WAITING", 
                "StateChangeReason": {
                    "Message": "Cluster ready after last step completed."
                }
            }, 
            "NormalizedInstanceHours": 30784, 
            "Id": "j-2L4YWO6XZPQTE", 
            "Name": "DS Training"
        }, 
        {
            "Status": {
                "Timeline": {
                    "EndDateTime": 1466003049.525, 
                    "CreationDateTime": 1466002991.478
                }, 
                "State": "TERMINATED_WITH_ERRORS", 
                "StateChangeReason": {
                    "Message": "The requested number of instances exceeds your EC2 quota", 
                    "Code": "VALIDATION_ERROR"
                }
            }, 
            "Norm

In [6]:
#!aws emr socks --cluster-id j-2L4YWO6XZPQTE --key-pair-file ~/datasci.ppk

** I figured out how to list the clusters but got stuck after that.

HW2.2: Analyze the performance of your Mappers, Combiners and Reducers using Counters

For this brief study the Input file will be one record (the next line only): 
foo foo quux labs foo bar quux

Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain.

In [7]:
!echo foo foo quux labs foo bar quux > WordCount22.txt

In [8]:
%%writefile vjs_wc_counter.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
 
WORD_RE = re.compile(r"[\w']+")
 
class vjsWordFreqCount(MRJob):
    def mapper(self, _, line):
        self.increment_counter('group', 'Num_mapper_calls', 1)
        for word in WORD_RE.findall(line):
            yield word.lower(), 1

    def reducer(self, word, counts):
        self.increment_counter('group', 'Num_reducer_calls', 1)
        yield word, sum(counts)

if __name__ == '__main__':
    vjsWordFreqCount.run()

Overwriting vjs_wc_counter.py


In [9]:
from vjs_wc_counter import vjsWordFreqCount
mr_job = vjsWordFreqCount(args=['WordCount22.txt'])
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access of the output 
    print runner.counters()

[{'group': {'Num_mapper_calls': 1, 'Num_reducer_calls': 4}}]


** There are 4 unique values (foo, quux, labs & bar) so while there is one mapper call (due to one line of data), it calls the reducer separately for each unique key.

Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many times the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

In [10]:
%%writefile vjsCC_issuecnt.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string
 
class vjsCCiss(MRJob):
    
    def mapper(self, _, line):
        self.increment_counter('group','Num_mapper_calls',1)
        line = line.lower()
        cid,grp,sprod,issue,_ = line.split(",",4)
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        issue = regex.sub(' ', issue.lower())     
        issue = issue.split()
        if grp != 'product':
            for word in issue:
                yield word,1
        del cid, grp, sprod, _
            
    def reducer(self,word,counts):
        self.increment_counter('group','Num_reducer_calls',1)
        yield word, sum(counts)

if __name__ == '__main__':
    vjsCCiss.run()

Overwriting vjsCC_issuecnt.py


In [11]:
from vjsCC_issuecnt import vjsCCiss
mr_job = vjsCCiss(args=['Consumer_Complaints.csv'])
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access of the output 
    print runner.counters()
    for line in runner.stream_output():
        print line.strip()


[{'group': {'Num_mapper_calls': 312913, 'Num_reducer_calls': 172}}]
"a"	3503
"account"	20681
"acct"	163
"action"	2505
"advance"	240
"advertising"	1193
"amount"	98
"amt"	71
"an"	2505
"and"	16448
"application"	8868
"applied"	139
"apply"	118
"apr"	3431
"arbitration"	168
"are"	3821
"atm"	2422
"attempts"	11848
"available"	274
"balance"	597
"bank"	202
"bankruptcy"	222
"being"	5663
"billing"	8158
"by"	5663
"can"	1999
"cancelling"	2795
"card"	4405
"cash"	240
"caused"	5663
"changes"	350
"charged"	976
"charges"	131
"checks"	75
"closing"	2795
"club"	12545
"collect"	11848
"collection"	1907
"communication"	6920
"company"	4858
"cont"	11848
"contact"	3053
"convenience"	75
"costs"	4350
"credit"	55251
"credited"	92
"customer"	2734
"d"	11848
"day"	71
"dealing"	1944
"debit"	2422
"debt"	19309
"decision"	2774
"decrease"	1149
"delay"	243
"delinquent"	1061
"deposits"	10555
"determination"	1490
"did"	139
"didn"	925
"disclosure"	5214
"disclosures"	64
"dispute"	904
"disputes"	6938
"embezzlement"	3276
"expect"	8

** Number of Mapper Calls: 312,913 - Number of Reducer Calls: 172

Perform a word count analysis of the Issue column of the Consumer Complaints Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, combiner counter, and Reducer Counter after completing your word count job.

In [12]:
%%writefile vjsCC_issuecnt2.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string
 
class vjsCCiss(MRJob):
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper),
            MRStep(combiner=self.combiner),
            MRStep(reducer=self.reducer)]
    
    def mapper(self, _, line):
        self.increment_counter('group','Num_mapper_calls',1)
        line = line.lower()
        cid,grp,sprod,issue,_ = line.split(",",4)
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        issue = regex.sub(' ', issue.lower())
        issue = issue.split()
        if grp != 'product':
            for word in issue:
                yield word, 1
        
    def combiner(self,word,counts):
        self.increment_counter('group','Num_combiner_calls',1)
        yield word, sum(counts)
            
    def reducer(self,word,counts):
        self.increment_counter('group','Num_reducer_calls',1)
        yield word, sum(counts)

if __name__ == '__main__':
    vjsCCiss.run()

Overwriting vjsCC_issuecnt2.py


In [13]:
from vjsCC_issuecnt2 import vjsCCiss
mr_job = vjsCCiss(args=['Consumer_Complaints.csv'])
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access of the output 
    print runner.counters()
    for line in runner.stream_output():
        print line.strip()


[{'group': {'Num_mapper_calls': 312913}}, {'group': {'Num_combiner_calls': 483}}, {'group': {'Num_reducer_calls': 172}}]
"a"	3503
"account"	20681
"acct"	163
"action"	2505
"advance"	240
"advertising"	1193
"amount"	98
"amt"	71
"an"	2505
"and"	16448
"application"	8868
"applied"	139
"apply"	118
"apr"	3431
"arbitration"	168
"are"	3821
"atm"	2422
"attempts"	11848
"available"	274
"balance"	597
"bank"	202
"bankruptcy"	222
"being"	5663
"billing"	8158
"by"	5663
"can"	1999
"cancelling"	2795
"card"	4405
"cash"	240
"caused"	5663
"changes"	350
"charged"	976
"charges"	131
"checks"	75
"closing"	2795
"club"	12545
"collect"	11848
"collection"	1907
"communication"	6920
"company"	4858
"cont"	11848
"contact"	3053
"convenience"	75
"costs"	4350
"credit"	55251
"credited"	92
"customer"	2734
"d"	11848
"day"	71
"dealing"	1944
"debit"	2422
"debt"	19309
"decision"	2774
"decrease"	1149
"delay"	243
"delinquent"	1061
"deposits"	10555
"determination"	1490
"did"	139
"didn"	925
"disclosure"	5214
"disclosures"	64
"disput

** Number of Mapper Calls: 312,913 - Number of Combiner Calls: 483 - Number of Reducer Calls: 172

HW2.2.1: 
Using a single reducer perform a sort of the words in decreasing order of word counts. Present the top 50 terms and their frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

HINT: You will need a second MRStep for the sort part. Step 1 will be the usual word count, while step 2 will be a sort step. Please use the Hadoop/MRJob framework to perform the sort. Please do NOT use any of the built-in sorts  from  python.


In [14]:
%%writefile vjsCC_issuetop50.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string
 
class vjsCCiss(MRJob):
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper,reducer=self.reducer),
            MRStep(mapper=self.mapper_sort,reducer=self.reducer_sort)]
    
    def mapper(self, _, line):
        line = line.lower()
        cid,grp,sprod,issue,_ = line.split(",",4)
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        issue = regex.sub(' ', issue.lower())
        issue = issue.split()
        if grp != 'product':
            for word in issue:
                yield (word, 1)
            
    def reducer(self, word, counts):
        yield word, sum(counts)
        
#MRStep2
    def mapper_sort(self, word, count):
        yield (count-1e15, word), None
        
    def reducer_sort(self, cntw, _):
        cnt, word = cntw
        yield word, int(abs(cnt + 1e15))
                
if __name__ == '__main__':
    vjsCCiss.run()

Overwriting vjsCC_issuetop50.py


In [15]:
from vjsCC_issuetop50 import vjsCCiss
mr_job = vjsCCiss(args=['Consumer_Complaints.csv', '--jobconf="mapred.reduce.tasks=1"'])
with mr_job.make_runner() as runner: 
    runner.run()
    cnt2 = 1
    for line in runner.stream_output():
        print cnt2, line.strip()
        cnt2 += 1
        if cnt2 > 50:
            break

1 "loan"	119630
2 "modification"	70487
3 "credit"	55251
4 "servicing"	36767
5 "report"	34903
6 "incorrect"	29133
7 "information"	29069
8 "on"	29069
9 "or"	22533
10 "account"	20681
11 "debt"	19309
12 "and"	16448
13 "opening"	16205
14 "club"	12545
15 "health"	12545
16 "not"	12353
17 "attempts"	11848
18 "collect"	11848
19 "cont"	11848
20 "d"	11848
21 "owed"	11848
22 "of"	10885
23 "my"	10731
24 "deposits"	10555
25 "withdrawals"	10555
26 "problems"	9484
27 "application"	8868
28 "to"	8401
29 "unable"	8178
30 "billing"	8158
31 "other"	7886
32 "disputes"	6938
33 "communication"	6920
34 "tactics"	6920
35 "reporting"	6559
36 "lease"	6337
37 "the"	6248
38 "being"	5663
39 "by"	5663
40 "caused"	5663
41 "funds"	5663
42 "low"	5663
43 "process"	5505
44 "disclosure"	5214
45 "verification"	5214
46 "managing"	5006
47 "company"	4858
48 "investigation"	4858
49 "s"	4858
50 "identity"	4729


Bottom 10 Values

In [16]:
%%writefile vjsCC_issuebot10.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string
 
class vjsCCiss(MRJob):
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper,reducer=self.reducer),
            MRStep(mapper=self.mapper_sort,reducer=self.reducer_sort)]
    
    def mapper(self, _, line):
        line = line.lower()
        cid,grp,sprod,issue,_ = line.split(",",4)
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        issue = regex.sub(' ', issue.lower())
        issue = issue.split()
        if grp != 'product':
            for word in issue:
                yield (word, 1)
            
    def reducer(self, word, counts):
        yield word, sum(counts)
        
#MRStep2
    def mapper_sort(self, word, count):
        yield ('%010d'%int(count), word), None
        
    def reducer_sort(self, cntw, _):
        cnt, word = cntw
        cnt = int(cnt)
        yield (word, cnt)
                
if __name__ == '__main__':
    vjsCCiss.run()

Overwriting vjsCC_issuebot10.py


In [17]:
from vjsCC_issuebot10 import vjsCCiss
mr_job = vjsCCiss(args=['Consumer_Complaints.csv', '--jobconf="mapred.reduce.tasks=1"'])
with mr_job.make_runner() as runner: 
    runner.run()
    cnt2 = 1
    for line in runner.stream_output():
        print cnt2, line.strip()
        cnt2 += 1
        if cnt2 > 10:
            break

1 "disclosures"	64
2 "missing"	64
3 "amt"	71
4 "day"	71
5 "checks"	75
6 "convenience"	75
7 "credited"	92
8 "payment"	92
9 "amount"	98
10 "apply"	118


HW2.2.2:   
Repeat HW2.2.1 using 3 reducers. Use the same code as in HW2.2.1  with just one modification 
to the command line: just add --jobconf mapred.reduce.tasks=3 as see presented here: 

    python HW2.2WordCount.py --jobconf mapred.reduce.tasks=3 oneLinerTextFile.txt

Describe what you see. Is this correct?

In [18]:
from vjsCC_issuetop50 import vjsCCiss
mr_job = vjsCCiss(args=['Consumer_Complaints.csv', '--jobconf="mapred.reduce.tasks=3"'])
with mr_job.make_runner() as runner: 
    runner.run()
    cnt2 = 1
    for line in runner.stream_output():
        print cnt2, line.strip()
        cnt2 += 1
        if cnt2 > 50:
            break

1 "loan"	119630
2 "modification"	70487
3 "credit"	55251
4 "servicing"	36767
5 "report"	34903
6 "incorrect"	29133
7 "information"	29069
8 "on"	29069
9 "or"	22533
10 "account"	20681
11 "debt"	19309
12 "and"	16448
13 "opening"	16205
14 "club"	12545
15 "health"	12545
16 "not"	12353
17 "attempts"	11848
18 "collect"	11848
19 "cont"	11848
20 "d"	11848
21 "owed"	11848
22 "of"	10885
23 "my"	10731
24 "deposits"	10555
25 "withdrawals"	10555
26 "problems"	9484
27 "application"	8868
28 "to"	8401
29 "unable"	8178
30 "billing"	8158
31 "other"	7886
32 "disputes"	6938
33 "communication"	6920
34 "tactics"	6920
35 "reporting"	6559
36 "lease"	6337
37 "the"	6248
38 "being"	5663
39 "by"	5663
40 "caused"	5663
41 "funds"	5663
42 "low"	5663
43 "process"	5505
44 "disclosure"	5214
45 "verification"	5214
46 "managing"	5006
47 "company"	4858
48 "investigation"	4858
49 "s"	4858
50 "identity"	4729


** We see the same results that we got using a single reducer. This is correct as the job loops through two sets of mapper-reducer. The key value pairs have been aggregated by the end of the first loop so the second pass acts as if there is a single reducer.

HW2.3: Shopping Cart Analysis
Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.
  
For this homework use the online browsing behavior dataset located at: 

       https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

Each line in this dataset represents a browsing session of a customer. 
On each line, each string of 8 characters represents the id of an item browsed during that session. 
The items are separated by spaces.

Here are the first few lines of the ProductPurchaseData 
FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 


Do some exploratory data analysis of this dataset guided by the following questions:. 

How many unique items are available from this supplier?

Using a single reducer: Report your findings: such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,   (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 


In [19]:
!curl -L https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0 -o Shop_Cart.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 3377k  100 3377k    0     0  1572k      0  0:00:02  0:00:02 --:--:-- 1572k


In [20]:
purchasedata = open('Shop_Cart.txt',"r+")
unq_items = []
for trip in purchasedata:
    items = trip.split()
    for item in items:
        if item not in unq_items:
            unq_items.append(item)
print 'Unique Item Count:', len(unq_items)

Unique Item Count: 12592


In [21]:
%%writefile vjs_wc_shcart.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string
 
class vjsCCiss(MRJob):
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper,reducer=self.reducer),
            MRStep(mapper=self.mapper_sort,reducer=self.reducer_sort)]
    
    def mapper(self, _, line):
        line = line.upper()
        items = line.split()
        for item in items:
            yield (item, 1)
            
    def reducer(self, word, counts):
        yield word, sum(counts)
        
#MRStep2
    def mapper_sort(self, word, count):
        yield (count-1e15, word), None
        
    def reducer_sort(self, cntw, _):
        cnt, word = cntw
        yield word, int(abs(cnt + 1e15))
                
if __name__ == '__main__':
    vjsCCiss.run()

Overwriting vjs_wc_shcart.py


In [22]:
%reload_ext autoreload
%autoreload 2

from vjs_wc_shcart import vjsCCiss
mr_job = vjsCCiss(args=['Shop_Cart.txt', '--jobconf="mapred.reduce.tasks=1"'])
with mr_job.make_runner() as runner: 
    runner.run()
    cnt = 1
    cnt2 = 0
    for line in runner.stream_output():
        if cnt < 51:
            print cnt, line.strip()
            cnt += 1
        cnt2 += 1
    print 'Unique Items:', cnt2

1 "DAI62779"	6667
2 "FRO40251"	3881
3 "ELE17451"	3875
4 "GRO73461"	3602
5 "SNA80324"	3044
6 "ELE32164"	2851
7 "DAI75645"	2736
8 "SNA45677"	2455
9 "FRO31317"	2330
10 "DAI85309"	2293
11 "ELE26917"	2292
12 "FRO80039"	2233
13 "GRO21487"	2115
14 "SNA99873"	2083
15 "GRO59710"	2004
16 "GRO71621"	1920
17 "FRO85978"	1918
18 "GRO30386"	1840
19 "ELE74009"	1816
20 "GRO56726"	1784
21 "DAI63921"	1773
22 "GRO46854"	1756
23 "ELE66600"	1713
24 "DAI83733"	1712
25 "FRO32293"	1702
26 "ELE66810"	1697
27 "SNA55762"	1646
28 "DAI22177"	1627
29 "FRO78087"	1531
30 "ELE99737"	1516
31 "ELE34057"	1489
32 "GRO94758"	1489
33 "FRO35904"	1436
34 "FRO53271"	1420
35 "SNA93860"	1407
36 "SNA90094"	1390
37 "GRO38814"	1352
38 "ELE56788"	1345
39 "GRO61133"	1321
40 "DAI88807"	1316
41 "ELE74482"	1316
42 "ELE59935"	1311
43 "SNA96271"	1295
44 "DAI43223"	1290
45 "ELE91337"	1289
46 "GRO15017"	1275
47 "DAI31081"	1261
48 "GRO81087"	1220
49 "DAI22896"	1219
50 "GRO85051"	1214
Unique Items: 12592


In [23]:
purchasedata = open('Shop_Cart.txt',"r+")
cntr = 0
for trip in purchasedata:
    cnt = 0
    items = trip.split()
    for item in items:
        cnt += 1
    if cnt > cntr:
        cntr = cnt
print 'Largest Basket Size:', cntr, 'items'

Largest Basket Size: 37 items


HW2.4. (Computationally prohibitive but then again Hadoop can handle this) Pairs

From a data mining perspective (and the aPriori algorihtm in particular), Support and Confidence are defined as follows:

       SUPPORT
       In data mining, the support value of X (where X is a collection of co-occurring items sometimes referred to as 
       an item-set. E.g., a basket or subset of a basket) with respect to T  (a transaction database where each 
       row is a transaction such as a basket of items that have been purchased)  is defined as the proportion 
       of transactions in the  database which contains  the item-set X. (a relative frequency of sorts) 

       CONFIDENCE 
       The confidence value of a rule, X ==>  Y (where X is a collection of cooccurring items and Y is generally 
       a single item. E.g., If Diapers and Beer then Cigars were also purchased), with respect to a set of transactions T, is the 
       proportion of the transactions that contains X which also contains Y. (Think of it as  tgePr(Y|X) )

       The pairs/stripes algorithm returns cooccurrence information that can be used directly to  calculate the confidence and support. 
       Note that confidence for pair X ==>  Y will  differ from the relative frequency that results from stripes when X occurs by itself in transactions.


Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a map-reduce program 
to find products which are frequently browsed together. Fix the support count (co-occurence count) to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) that have a support count of 100 or more.

List the top 50 product pairs with corresponding support count (aka frequency), and relative frequency or support (number of records where they coccur, the number of records where they coccur/the number of baskets in the dataset)  in decreasing order of support  for frequent (100>count) itemsets of size 2. 

Use the Pairs pattern  to  extract these frequent itemsets of size 2. Free free to use combiners if they bring value. Instrument your code with counters to count the number of times your mapper, combiner and reducers are called.  

Please output records of the following form for the top 50 pairs (itemsets of size 2): 

      item1, item2, support count, support  (OPTIONAL Feel free to add in confidence level also)



Fix the ordering of the pairs lexicographically (left to right), 
and break ties in support (between pairs, if any exist) 
by taking the first ones in lexicographically increasing order. 

Report  the compute time for the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)
Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts.


In [24]:
%%writefile vjs_wc_shcart_prs.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string

tot = {}
 
class vjsCCiss(MRJob):
        
    def steps(self):
        return [
            MRStep(mapper=self.mapper,combiner=self.combiner,reducer=self.reducer),
            MRStep(mapper=self.mapper_sort,reducer=self.reducer_sort)]
    
    def mapper(self, _, line):
        self.increment_counter('group', 'Mapper', 1)
        line = line.upper()
        items = list(set(line.split()))
        items.sort()
        
        for index, item1 in enumerate(items):
            for item2 in items[index + 1:]:
                yield (item1,item2),1
        yield ('total',''),1
                
    def combiner(self,words,counts):
        self.increment_counter('group', 'Combiner', 1)
        yield words, sum(counts)
        
    def reducer(self, words, counts):
        self.increment_counter('group', 'Reducer', 1)
        count = sum(counts)
        if len(words[1]) < 1:
            tot['total'] = count
        if count > 99:
            yield (words, count)
    
    def mapper_sort(self,words,count):
        yield(count-1e15,words),None
        
    def reducer_sort(self,words,_):
        t_cnt = float(tot["total"])
        c,w = words
        c2 = c + 1e15
        yield w,(c2, (c2/t_cnt))

if __name__ == '__main__':
    vjsCCiss.run()

Overwriting vjs_wc_shcart_prs.py


In [25]:
%reload_ext autoreload
%autoreload 2

from vjs_wc_shcart_prs import vjsCCiss
mr_job = vjsCCiss(args=['Shop_Cart.txt', '--jobconf="mapred.reduce.tasks=3"'])
with mr_job.make_runner() as runner: 
    runner.run()
    print runner.counters()
    for line in runner.stream_output():
        print line.strip()

[{'group': {'Mapper': 31101, 'Reducer': 877096, 'Combiner': 1026709}}, {}]
["total", ""]	[31101.0, 1.0]
["DAI62779", "ELE17451"]	[1592.0, 0.05118806469245362]
["FRO40251", "SNA80324"]	[1412.0, 0.04540046943828173]
["DAI75645", "FRO40251"]	[1254.0, 0.04032024693739751]
["FRO40251", "GRO85051"]	[1213.0, 0.039001961351725026]
["DAI62779", "GRO73461"]	[1139.0, 0.03662261663612103]
["DAI75645", "SNA80324"]	[1130.0, 0.03633323687341243]
["DAI62779", "FRO40251"]	[1070.0, 0.03440403845535513]
["DAI62779", "SNA80324"]	[923.0, 0.029677502331114755]
["DAI62779", "DAI85309"]	[918.0, 0.029516735796276648]
["ELE32164", "GRO59710"]	[911.0, 0.029291662647503297]
["DAI62779", "DAI75645"]	[882.0, 0.02835921674544227]
["FRO40251", "GRO73461"]	[882.0, 0.02835921674544227]
["DAI62779", "ELE92920"]	[877.0, 0.02819845021060416]
["FRO40251", "FRO92469"]	[835.0, 0.026848011317964052]
["DAI62779", "ELE32164"]	[832.0, 0.026751551397061188]
["DAI75645", "GRO73461"]	[712.0, 0.022893154560946594]
["DAI43223", "ELE3

HW2.5: Stripes
Repeat 2.4 using the stripes design pattern for finding co-occuring pairs (and out.

Report  the compute times for stripes job versus the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)

Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts. Discuss the differences in these counts between the Pairs and Stripes jobs


In [26]:
%%writefile vjs_wc_shcart_strp.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string
from collections import Counter

stripe = {}
 
class vjsCCiss(MRJob):
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper,combiner=self.combiner,reducer=self.reducer)]
       
    def mapper(self, _, line):
        self.increment_counter('group', 'Mapper', 1)
        line = line.upper()
        items = list(set(line.split()))
        items.sort()
        
        for index, item1 in enumerate(items):
            cnt = Counter()
            for item2 in items[index + 1:]:
                cnt[item2] += 1
            yield item1, cnt
        yield 'total',{'total':1}
                
    def combiner(self,skus,cntr):
        self.increment_counter('group', 'Combiner', 1)
        cnt = Counter()
        for c in cntr:
            for item2, count in c.items():
                cnt[item2] += count
        yield skus, cnt 
        
    def reducer(self, skus, cntr):
        self.increment_counter('group', 'Reducer', 1)
        cnt = Counter()
        for c in cntr:
            for item2, count in c.items():
                cnt[item2] += count
            
        if skus == 'total':
            stripe[skus] = cntr[skus]
            return
            
        for item2, count in cntr.items():
            if count > 99:
                yield (skus, item2), count

if __name__ == '__main__':
    vjsCCiss.run()

Overwriting vjs_wc_shcart_strp.py


In [27]:
%reload_ext autoreload
%autoreload 2

from vjs_wc_shcart_strp import vjsCCiss
mr_job = vjsCCiss(args=['Shop_Cart.txt', '--jobconf="mapred.reduce.tasks=2"'])
with mr_job.make_runner() as runner: 
    runner.run()
    print runner.counters()
    for line in runner.stream_output():
        print line.strip()

AttributeError: 'generator' object has no attribute 'items'

I ran out of time trying to fix this - but I'm close.